# 通过净利润的思想来挑选股票

In [1]:
import pandas as pd
import numpy as np
import pdb
import time

import datetime

import sys
sys.path.append("../utils/")
import token_util
pro = token_util.set_token()

*获取股票列表*

In [2]:
stock_list = pd.read_csv("../data_pulled/stock_date_delta_priceNone.csv")
print("stock_list.shape: ", stock_list.shape)
print("")
stock_list.head()

stock_list.shape:  (3093, 2)



,ts_code,name
0,000002.SZ,万科A
1,000005.SZ,世纪星源
2,000006.SZ,深振业A
3,000007.SZ,全新好
4,000008.SZ,神州高铁


### 成长股
三年内年净利润增长率最少大于10%，平均大于25% <br>
毛利率三年平均>40% <br>
净利率三年平均>20% <br>

*获取数据*

In [3]:
def get_data(ts_code, time_list):
    for i in range(len(time_list)):

        ret = pro.income(ts_code=ts_code, period='20'+str(time_list[i])+'1231', fields='ts_code,end_date,n_income,\
                         revenue,oper_cost')
        rets = pd.concat([rets, ret], ignore_index=True) if i != 0 else ret

    rets = rets.drop_duplicates()[-4:].reset_index(drop=True)
    rets['mao_li'] = (rets['revenue']-rets['oper_cost'])/rets['revenue']
    rets['jing_li'] = rets['n_income']/rets['revenue']
    
    return rets.drop(['oper_cost', 'revenue'], axis=1)

*进行筛选*

In [4]:
candidate_list = []
time_list = [15, 16, 17, 18, 19, 20]
times = 0
starttime = datetime.datetime.now()
len_stock = len(stock_list)

while True:
    
    # 发生中断后，防止遗漏
    start_index = 0
    
    for cur_index in range(start_index, len_stock):
        
        start_index = cur_index
        
        try:
            data = get_data(stock_list["ts_code"][cur_index], time_list)
        except:
            time.sleep(5)

        times += 1
        pass_data = True
        in_come_list = []

        maoli_mean = data['mao_li'][1:].mean()
        jingli_mean = data['jing_li'][1:].mean()

        if maoli_mean >= 0.4 and jingli_mean >= 0.2:

            for i in range(1, len(data)):
                n_income_last = data.iloc[i-1]["n_income"]
                n_income_cur = data.iloc[i]["n_income"]
                income_rate = (n_income_cur - n_income_last) / n_income_last
                in_come_list.append(income_rate)

                if n_income_cur > 0 and n_income_cur > n_income_last and income_rate >= 0.10:
                    continue
                else:
                    pass_data = False
                    break
        else:
            pass_data = False
        
        if len(in_come_list) == 3 and pass_data is True and np.mean(in_come_list) >= 0.25:
            candidate_list.append(stock_list["ts_code"][cur_index])
            
        if cur_index != 0 and cur_index % 100 == 0:
            endtime = datetime.datetime.now()
            print("task accomplish: {} %, cost time : {} minutes".format(round((cur_index+1)/len_stock, 2), \
                                            round((endtime - starttime).seconds / 60, 2)))
            
    if cur_index >= len_stock - 1:
        endtime = datetime.datetime.now()
        print("finish task !! total time : {} minutes".format(round((endtime - starttime).seconds / 60, 2)))
        break

task accomplish: 0.03 %, cost time : 2.37 minutes
task accomplish: 0.06 %, cost time : 5.22 minutes
task accomplish: 0.1 %, cost time : 7.6 minutes
task accomplish: 0.13 %, cost time : 10.47 minutes
task accomplish: 0.16 %, cost time : 12.87 minutes
task accomplish: 0.19 %, cost time : 15.75 minutes
task accomplish: 0.23 %, cost time : 18.1 minutes
task accomplish: 0.26 %, cost time : 20.92 minutes
task accomplish: 0.29 %, cost time : 23.23 minutes
task accomplish: 0.32 %, cost time : 26.03 minutes
task accomplish: 0.36 %, cost time : 28.43 minutes
task accomplish: 0.39 %, cost time : 31.27 minutes
task accomplish: 0.42 %, cost time : 33.55 minutes
task accomplish: 0.45 %, cost time : 36.33 minutes
task accomplish: 0.49 %, cost time : 38.65 minutes
task accomplish: 0.52 %, cost time : 41.52 minutes
task accomplish: 0.55 %, cost time : 43.83 minutes
task accomplish: 0.58 %, cost time : 46.57 minutes
task accomplish: 0.61 %, cost time : 48.93 minutes
task accomplish: 0.65 %, cost time : 

In [5]:
len(candidate_list)

71

In [6]:
candidate_list

['000568.SZ',
 '000661.SZ',
 '000813.SZ',
 '000858.SZ',
 '002001.SZ',
 '002040.SZ',
 '002099.SZ',
 '002195.SZ',
 '002372.SZ',
 '002425.SZ',
 '002619.SZ',
 '002677.SZ',
 '002821.SZ',
 '002901.SZ',
 '002943.SZ',
 '300003.SZ',
 '300004.SZ',
 '300005.SZ',
 '300006.SZ',
 '300007.SZ',
 '300008.SZ',
 '300009.SZ',
 '300010.SZ',
 '300016.SZ',
 '300251.SZ',
 '300294.SZ',
 '300326.SZ',
 '300365.SZ',
 '300396.SZ',
 '300450.SZ',
 '300459.SZ',
 '300482.SZ',
 '300506.SZ',
 '300529.SZ',
 '300559.SZ',
 '300595.SZ',
 '300628.SZ',
 '300630.SZ',
 '300634.SZ',
 '300642.SZ',
 '300653.SZ',
 '300685.SZ',
 '300699.SZ',
 '300725.SZ',
 '300732.SZ',
 '300747.SZ',
 '300751.SZ',
 '300753.SZ',
 '300766.SZ',
 '600054.SH',
 '600167.SH',
 '600276.SH',
 '600368.SH',
 '600516.SH',
 '600603.SH',
 '600639.SH',
 '600771.SH',
 '603037.SH',
 '603338.SH',
 '603357.SH',
 '603387.SH',
 '603393.SH',
 '603429.SH',
 '603505.SH',
 '603568.SH',
 '603583.SH',
 '603589.SH',
 '603638.SH',
 '603658.SH',
 '603666.SH',
 '603707.SH']

In [7]:
stock_list[(stock_list["ts_code"] == stock_list["ts_code"][cur_index])]

,ts_code,name
3092,603999.SH,读者传媒
